In [ ]:
pip install ultralytics

In [ ]:
from google.colab import files
import zipfile

uploaded_model = files.upload()

In [ ]:
from google.colab import files
import zipfile

uploaded_model = files.upload()

In [ ]:
from google.colab import files
import zipfile

# Upload the model file
uploaded_model = files.upload()  # 'last.pt'

uploaded_images = files.upload()  # 'google_images'

In [ ]:
import zipfile
import os

with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

print("Contents of '/content/dataset':", os.listdir('/content/dataset'))

In [ ]:
import os

print("Contents of '/content/dataset/dataset':", os.listdir('/content/dataset/dataset'))

In [ ]:
pip install easyocr

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# YOLO model for detecting license plates
yolo_model = YOLO('yolov8n.pt')

# Step 1: Detect license plate in images and prepare character data for OCR training
def prepare_character_data(labels_df, images_path):
    character_data = []  # Store character images and labels

    for _, row in labels_df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if not os.path.exists(image_path):
            continue
        true_text = row['License Plate']

        # Get license plate bounding box using YOLO
        results = yolo_model.predict(image_path)
        boxes = results[0].boxes.xyxy.cpu().numpy()  # Get bounding box coordinates

        # Assume only one plate per image; get first box
        if len(boxes) > 0:
            x1, y1, x2, y2 = map(int, boxes[0][:4])  # First detected box
            license_plate = cv2.imread(image_path)[y1:y2, x1:x2]

            # Process each character
            for i, char in enumerate(true_text):
                char_img = preprocess_char(license_plate, i, len(true_text))  # Function for char cropping
                character_data.append((char_img, char))

    return character_data

# Preprocess character images for consistent sizing and normalization
def preprocess_char(license_plate, index, total_chars):
    plate_height, plate_width = license_plate.shape[:2]
    char_width = plate_width // total_chars  # Approximate width per character

    x1, x2 = index * char_width, (index + 1) * char_width
    char_img = license_plate[:, x1:x2]  # Crop character
    char_img = cv2.resize(char_img, (32, 32))  # Resize to (32, 32) for CNN
    char_img = cv2.cvtColor(char_img, cv2.COLOR_BGR2GRAY)
    char_img = char_img / 255.0  # Normalize

    return char_img

# Prepare character dataset
character_data = prepare_character_data(labels_df, images_path)
character_images, character_labels = zip(*character_data)
character_images = np.array(character_images).reshape(-1, 32, 32, 1)
character_labels = np.array(character_labels)

# Encode character labels
label_encoder = {label: idx for idx, label in enumerate(set(character_labels))}
character_labels_encoded = np.array([label_encoder[char] for char in character_labels])

# Split character data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(character_images, character_labels_encoded, test_size=0.2, random_state=42)

# Step 2: Build a CNN model for character recognition
def create_character_recognition_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(label_encoder), activation='softmax')  # Output size based on unique chars
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train the character recognition model
char_recognition_model = create_character_recognition_model()
char_recognition_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Step 3: Evaluate model on test set and display results
def display_image_with_ocr_results(image_path, boxes, true_text, ocr_text):
    image = cv2.imread(image_path)
    for (x1, y1, x2, y2), char in zip(boxes, ocr_text):
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, char, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Evaluate on test data
for _, row in labels_df.iterrows():
    image_path = os.path.join(images_path, row['Image Name'])
    true_text = row['License Plate']

    # Detect license plate
    results = yolo_model.predict(image_path)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    if len(boxes) > 0:
        x1, y1, x2, y2 = map(int, boxes[0][:4])
        license_plate = cv2.imread(image_path)[y1:y2, x1:x2]

        # Predict each character using trained OCR model
        ocr_text = []
        for i in range(len(true_text)):
            char_img = preprocess_char(license_plate, i, len(true_text))
            char_img = char_img.reshape(1, 32, 32, 1)
            predicted_label = np.argmax(char_recognition_model.predict(char_img))
            predicted_char = list(label_encoder.keys())[list(label_encoder.values()).index(predicted_label)]
            ocr_text.append(predicted_char)

        # Display results
        display_image_with_ocr_results(image_path, [boxes[0]], true_text, ocr_text)

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2
import easyocr
import yaml
import matplotlib.pyplot as plt

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# Ensure that the DataFrame contains at least 300 images
if len(labels_df) < 300:
    raise ValueError("The manual_labels.csv file must contain at least 300 images.")

# Use the first 240 images as training data and the remaining 60 as testing data
train_labels = labels_df.iloc[:240]
test_labels = labels_df.iloc[240:]

# Verify that the images exist in the dataset
labeled_images = train_labels['Image Name'].tolist() + test_labels['Image Name'].tolist()
existing_images = os.listdir(images_path)

# Filter to keep only existing images in labeled data
train_labels = train_labels[train_labels['Image Name'].isin(existing_images)]
test_labels = test_labels[test_labels['Image Name'].isin(existing_images)]

# Separate unlabeled images
unlabeled_images = [img for img in existing_images if img not in labeled_images]

# Split unlabeled images into 80% train and 20% test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move labeled images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)

# Move unlabeled images to their respective train and test directories
def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Save the updated labels for train and test sets
train_labels.to_csv(os.path.join(train_dir, 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(test_dir, 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to display images with bounding boxes and OCR text
def display_image_with_boxes(image_path, boxes, text):
    image = cv2.imread(image_path)
    for box, t in zip(boxes, text):
        x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(image, t, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add OCR text

    # Convert BGR to RGB for matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Initialize results list to save predictions
results_list = []

# Evaluating and displaying OCR predictions on the test set
def evaluate_and_save_ocr(test_labels, model):
    for _, row in test_labels.iterrows():
        image_path = os.path.join(test_dir, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'  # Expected license plate text or 'None'

        # Get model predictions for bounding boxes
        results = model.predict(image_path)

        # Assuming results contains bounding box coordinates
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates

        # Initialize a list to hold the predicted texts
        predicted_texts = []

        # Loop through bounding boxes to extract text using EasyOCR
        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]  # Crop the image to the bounding box

            # Use EasyOCR to read the text in the cropped image
            ocr_results = reader.readtext(cropped_image)
            extracted_text = " ".join([text[1] for text in ocr_results])  # Join detected text
            predicted_texts.append(extracted_text)  # Add extracted text to the list

        # Draw bounding boxes and OCR text on the image
        display_image_with_boxes(image_path, bounding_boxes, predicted_texts)

        # Append results for saving
        results_list.append({
            'Image Name': row['Image Name'],
            'predicted_text': " ".join(predicted_texts),  # Combine all predicted texts
            'actual_text': true_text
        })

    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('/content/test_predictions.csv', index=False)
    print("Predicted readings saved to /content/test_predictions.csv")

# Load the YOLO model for prediction
yolo_model = YOLO('last.pt')

# Fine-tune the model using the training data (train_labels)
# You can add your training code here if needed.

# Run evaluation and save predictions
evaluate_and_save_ocr(test_labels, yolo_model)

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2
import easyocr
import yaml
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# Ensure that the DataFrame contains at least 300 images
if len(labels_df) < 300:
    raise ValueError("The manual_labels.csv file must contain at least 300 images.")

# Use the first 240 images as training data and the remaining 60 as testing data
train_labels = labels_df.iloc[:240]
test_labels = labels_df.iloc[240:]

# Verify that the images exist in the dataset
labeled_images = train_labels['Image Name'].tolist() + test_labels['Image Name'].tolist()
existing_images = os.listdir(images_path)

# Filter to keep only existing images in labeled data
train_labels = train_labels[train_labels['Image Name'].isin(existing_images)]
test_labels = test_labels[test_labels['Image Name'].isin(existing_images)]

# Separate unlabeled images
unlabeled_images = [img for img in existing_images if img not in labeled_images]

# Split unlabeled images into 80% train and 20% test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move labeled images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)

# Move unlabeled images to their respective train and test directories
def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Save the updated labels for train and test sets
train_labels.to_csv(os.path.join(train_dir, 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(test_dir, 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to display images with bounding boxes and OCR text
def display_image_with_boxes(image_path, boxes, text):
    image = cv2.imread(image_path)
    for box, t in zip(boxes, text):
        x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(image, t, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add OCR text

    # Convert BGR to RGB for matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Initialize results list to save predictions and accuracy count
results_list = []
correct_predictions = 0

# Evaluating and displaying OCR predictions on the test set
def evaluate_and_save_ocr(test_labels, model):
    global correct_predictions  # Use global variable for correct predictions count
    for _, row in test_labels.iterrows():
        image_path = os.path.join(test_dir, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'  # Expected license plate text or 'None'

        # Get model predictions for bounding boxes
        results = model.predict(image_path)

        # Assuming results contains bounding box coordinates
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates

        # Initialize a list to hold the predicted texts
        predicted_texts = []

        # Loop through bounding boxes to extract text using EasyOCR
        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]  # Crop the image to the bounding box

            # Use EasyOCR to read the text in the cropped image
            ocr_results = reader.readtext(cropped_image)
            extracted_text = " ".join([text[1] for text in ocr_results])  # Join detected text
            predicted_texts.append(extracted_text)  # Add extracted text to the list

        # Combine all OCR results as a single prediction
        combined_predicted_text = " ".join(predicted_texts)

        # Compare prediction with actual text for accuracy calculation
        if combined_predicted_text.strip().upper() == true_text.strip().upper():
            correct_predictions += 1

        # Draw bounding boxes and OCR text on the image
        display_image_with_boxes(image_path, bounding_boxes, predicted_texts)

        # Append results for saving
        results_list.append({
            'Image Name': row['Image Name'],
            'predicted_text': combined_predicted_text,
            'actual_text': true_text
        })

    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('/content/test_predictions.csv', index=False)
    print("Predicted readings saved to /content/test_predictions.csv")

# Load the YOLO model for prediction
yolo_model = YOLO('last.pt')

evaluate_and_save_ocr(test_labels, yolo_model)

In [ ]:
import os
import shutil
import pandas as pd
import cv2
import easyocr
import numpy as np
from ultralytics import YOLO

# Define constants
IMAGES_PATH = '/content/dataset/dataset/google_images'
OUTPUT_DIR = '/content/processed_data'
MANUAL_LABELS_PATH = '/content/manual_labels.csv'
TRAIN_PREDICTIONS_PATH = '/content/train_predictions.csv'
TEST_PREDICTIONS_PATH = '/content/test_predictions.csv'
TRAIN_SIZE = 240
TEST_SIZE = 60

# Load manual labels
labels_df = pd.read_csv(MANUAL_LABELS_PATH)

# Ensure at least 300 images
if len(labels_df) < 300:
    raise ValueError("The manual_labels.csv file must contain at least 300 images.")

# Split labels into training and testing
train_labels = labels_df.iloc[:TRAIN_SIZE]
test_labels = labels_df.iloc[TRAIN_SIZE:TRAIN_SIZE + TEST_SIZE]

# Verify existing images
existing_images = os.listdir(IMAGES_PATH)
labeled_images = train_labels['Image Name'].tolist() + test_labels['Image Name'].tolist()

# Filter labels for existing images
train_labels = train_labels[train_labels['Image Name'].isin(existing_images)]
test_labels = test_labels[test_labels['Image Name'].isin(existing_images)]

# Prepare output directories
os.makedirs(os.path.join(OUTPUT_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'test'), exist_ok=True)

# Copy images to respective directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, os.path.join(OUTPUT_DIR, 'train'))
copy_images(test_labels, os.path.join(OUTPUT_DIR, 'test'))

# Save updated labels
train_labels.to_csv(os.path.join(OUTPUT_DIR, 'train', 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(OUTPUT_DIR, 'test', 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to preprocess images
def preprocess_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_img)
    blurred_img = cv2.GaussianBlur(clahe_img, (5, 5), 0)
    return blurred_img

# Additional preprocessing for cropped images
def enhance_cropped_image(cropped_img):
    # Convert to grayscale and apply binary threshold
    gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)
    morph_img = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel)

    return morph_img

# Function to log mismatches for learning updates
def log_mismatch(predicted_text, actual_text):
    with open('mismatch_log.txt', 'a') as log_file:
        log_file.write(f"Predicted: {predicted_text}, Actual: {actual_text}\n")

# Custom loss calculation based on the difference between predicted and actual text
def calculate_penalty(predicted_text, actual_text):
    if predicted_text != actual_text:
        # Count the character differences as a penalty
        penalty = len(set(actual_text) - set(predicted_text)) + len(set(predicted_text) - set(actual_text))
        # Log the mismatch for learning
        log_mismatch(predicted_text, actual_text)
        return penalty
    return 0

# Evaluate and save OCR predictions with custom loss adjustment
def evaluate_and_save_ocr(labels_df, output_file):
    results_list = []
    total_loss = 0

    for _, row in labels_df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'
        results = yolo_model.predict(image_path)
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()

        predicted_texts = []

        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]
            processed_image = preprocess_image(cropped_image)

            # Enhance the cropped image before OCR
            enhanced_image = enhance_cropped_image(cropped_image)

            # Use EasyOCR to read the text
            ocr_results = reader.readtext(enhanced_image)
            filtered_results = [text[1] for text in ocr_results if text[2] > 0.5]

            extracted_text = " ".join(filtered_results)
            predicted_texts.append(extracted_text)

        combined_predicted_text = " ".join(predicted_texts)

        # Custom loss adjustment
        penalty = calculate_penalty(combined_predicted_text, true_text)
        total_loss += penalty  # Accumulate total loss

        results_list.append({
            'Image Name': row['Image Name'],
            'actual_text': true_text,
            'predicted_text': combined_predicted_text,
            'penalty': penalty  # Include penalty in results
        })

    results_df = pd.DataFrame(results_list)
    results_df.to_csv(output_file, index=False)
    print(f"Predicted readings saved to {output_file}")

    # Print the total loss for this evaluation
    print(f"Total loss for the evaluation: {total_loss}")

# Load YOLO model
yolo_model = YOLO('last.pt')

# Evaluate predictions
evaluate_and_save_ocr(train_labels, TRAIN_PREDICTIONS_PATH)
evaluate_and_save_ocr(test_labels, TEST_PREDICTIONS_PATH)

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import os
import shutil
import pandas as pd
import cv2
import easyocr
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from ultralytics import YOLO
from sklearn.utils import shuffle

# Define constants
IMAGES_PATH = '/content/dataset/dataset/google_images'
OUTPUT_DIR = '/content/processed_data'
MANUAL_LABELS_PATH = '/content/manual_labels.csv'
TRAIN_PREDICTIONS_PATH = '/content/train_predictions.csv'
CHARACTER_MODEL_PATH = '/content/character_model.h5'
TRAIN_SIZE = 240
TEST_SIZE = 60

# Load manual labels
labels_df = pd.read_csv(MANUAL_LABELS_PATH)

# Ensure at least 300 images
if len(labels_df) < 300:
    raise ValueError("The manual_labels.csv file must contain at least 300 images.")

# Shuffle the labels dataframe
labels_df = shuffle(labels_df)

# Split labels into training and testing
train_labels = labels_df.iloc[:TRAIN_SIZE]
test_labels = labels_df.iloc[TRAIN_SIZE:TRAIN_SIZE + TEST_SIZE]

# Verify existing images
existing_images = os.listdir(IMAGES_PATH)
train_labels = train_labels[train_labels['Image Name'].isin(existing_images)]

# Prepare output directories
os.makedirs(os.path.join(OUTPUT_DIR, 'train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'test'), exist_ok=True)

# Copy images to respective directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, os.path.join(OUTPUT_DIR, 'train'))

# Save updated labels
train_labels.to_csv(os.path.join(OUTPUT_DIR, 'train', 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to preprocess images
def preprocess_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_img)
    blurred_img = cv2.GaussianBlur(clahe_img, (5, 5), 0)
    return blurred_img

# Function to log mismatches
def log_mismatch(predicted_text, actual_text):
    with open('mismatch_log.txt', 'a') as log_file:
        log_file.write(f"Predicted: {predicted_text}, Actual: {actual_text}\n")

# Function to calculate penalty
def calculate_penalty(predicted_text, actual_text):
    if predicted_text != actual_text:
        penalty = len(set(actual_text) - set(predicted_text)) + len(set(predicted_text) - set(actual_text))
        log_mismatch(predicted_text, actual_text)
        return penalty
    return 0

# Load YOLO model
yolo_model = YOLO('last.pt')

# Evaluate and save OCR predictions
def evaluate_and_save_ocr(labels_df, output_file):
    results_list = []

    for _, row in labels_df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'
        results = yolo_model.predict(image_path)
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()

        predicted_texts = []

        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]
            enhanced_image = preprocess_image(cropped_image)

            # Use EasyOCR to read the text
            ocr_results = reader.readtext(enhanced_image)
            filtered_results = [text[1] for text in ocr_results if text[2] > 0.5]

            extracted_text = " ".join(filtered_results).replace(" ", "")
            predicted_texts.append(extracted_text)

        combined_predicted_text = " ".join(predicted_texts)

        # Custom loss adjustment
        penalty = calculate_penalty(combined_predicted_text, true_text)

        results_list.append({
            'Image Name': row['Image Name'],
            'actual_text': true_text,
            'predicted_text': combined_predicted_text,
            'penalty': penalty
        })

    results_df = pd.DataFrame(results_list)
    results_df.to_csv(output_file, index=False)
    print(f"Predicted readings saved to {output_file}")

# Prepare character data and labels based on bounding box extraction
character_data = []
character_labels = []

# Function to extract character images from the license plate image
def extract_character_image(cropped_img, char, size=(28, 28)):
    # Convert to grayscale
    gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours and isolate character
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        char_image = binary_img[y:y + h, x:x + w]

        # Check if character size is adequate
        if char_image.shape[1] > 5 and char_image.shape[0] > 5:
            resized_img = cv2.resize(char_image, size)
            return resized_img

    return None

# Process images to create character data and labels for training
def prepare_character_data_and_labels(labels_df):
    for _, row in labels_df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else None
        if true_text:
            results = yolo_model.predict(image_path)
            bounding_boxes = results[0].boxes.xyxy.cpu().numpy()
            for box in bounding_boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]

                for character in true_text:
                    char_image = extract_character_image(cropped_image, character)
                    if char_image is not None:
                        character_data.append(char_image)
                        character_labels.append(character)

# Call the function to prepare data
prepare_character_data_and_labels(train_labels)

# Shuffle character data and labels
character_data, character_labels = shuffle(np.array(character_data), character_labels)

# Encode labels as integers
label_encoder = LabelEncoder()
character_labels_encoded = label_encoder.fit_transform(character_labels)

# Ensure character_data is of type float32 and scaled between 0 and 1
character_data = np.array(character_data, dtype='float32') / 255.0

# Reshape character_data to fit the model input
character_data = character_data.reshape(-1, 28, 28, 1)  # Grayscale images

# Check if character_data and character_labels_encoded have the same length
if len(character_data) != len(character_labels_encoded):
    raise ValueError("Mismatch in number of character images and labels.")

# Data augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # Typically, we don't flip characters
    fill_mode='nearest'
)

# Prepare the data generator for training without validation
train_generator = datagen.flow(character_data, character_labels_encoded,
                                batch_size=32)

# Create the model with the correct number of classes
num_classes = len(label_encoder.classes_)
character_model = Sequential([
    Input(shape=(28, 28, 1)),  # Change input shape to (28, 28, 1) for grayscale
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
character_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
character_model.fit(train_generator, epochs=10)

# Save the model
character_model.save(CHARACTER_MODEL_PATH)

# Evaluate and save OCR predictions
evaluate_and_save_ocr(test_labels, TRAIN_PREDICTIONS_PATH)

In [ ]:
import os
import shutil
import pandas as pd
import cv2
import easyocr
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Define constants
IMAGES_PATH = '/content/dataset/dataset/google_images'
OUTPUT_DIR = '/content/processed_data'
MANUAL_LABELS_PATH = '/content/manual_labels.csv'
CHARACTER_MODEL_PATH = '/content/character_model.h5'
TRAIN_SIZE = 240
TEST_SIZE = 60

# Load manual labels
labels_df = pd.read_csv(MANUAL_LABELS_PATH)

# Ensure at least 300 images
if len(labels_df) < 300:
    raise ValueError("The manual_labels.csv file must contain at least 300 images.")

# Shuffle and split the labels
labels_df = shuffle(labels_df)
train_labels = labels_df.iloc[:TRAIN_SIZE]

# Verify existing images
existing_images = os.listdir(IMAGES_PATH)
train_labels = train_labels[train_labels['Image Name'].isin(existing_images)]

# Prepare output directories
os.makedirs(os.path.join(OUTPUT_DIR, 'train'), exist_ok=True)

# Copy images to the train directory
for _, row in train_labels.iterrows():
    image_path = os.path.join(IMAGES_PATH, row['Image Name'])
    if os.path.exists(image_path):
        shutil.copy(image_path, os.path.join(OUTPUT_DIR, 'train'))

# Save train labels
train_labels.to_csv(os.path.join(OUTPUT_DIR, 'train', 'labels.csv'), index=False)

# Initialize YOLO model and EasyOCR reader
yolo_model = YOLO('last.pt')
reader = easyocr.Reader(['en'])

# Function to preprocess images
def preprocess_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_img

# Extract character data for training
character_data = []
character_labels = []

def extract_character_data_and_labels(labels_df):
    for _, row in labels_df.iterrows():
        image_path = os.path.join(IMAGES_PATH, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else None
        if true_text:
            results = yolo_model.predict(image_path)
            bounding_boxes = results[0].boxes.xyxy.cpu().numpy()
            for box in bounding_boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]

                for character in true_text:
                    resized_image = preprocess_image(cropped_image)
                    resized_image = cv2.resize(resized_image, (28, 28))
                    character_data.append(resized_image)
                    character_labels.append(character)

extract_character_data_and_labels(train_labels)

# Shuffle the data
character_data, character_labels = shuffle(np.array(character_data), character_labels)

# Encode labels
label_encoder = LabelEncoder()
character_labels_encoded = label_encoder.fit_transform(character_labels)

# Normalize character data
character_data = np.expand_dims(character_data, axis=-1) / 255.0

# Model architecture
num_classes = len(label_encoder.classes_)
character_model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile and train
character_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1)
train_generator = datagen.flow(character_data, character_labels_encoded, batch_size=32)

# Display images during training
for i in range(10):
    plt.imshow(character_data[i].squeeze(), cmap='gray')
    plt.title(f"Character: {label_encoder.inverse_transform([character_labels_encoded[i]])[0]}")
    plt.show()

character_model.fit(train_generator, epochs=10)

# Save the model
character_model.save(CHARACTER_MODEL_PATH)

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2
import easyocr
import yaml
import matplotlib.pyplot as plt
import difflib

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# Get the image file names from the labels
labeled_images = labels_df['Image Name'].tolist()

# Separate labeled and unlabeled images
all_images = os.listdir(images_path)
unlabeled_images = [img for img in all_images if img not in labeled_images]

# Split labeled images (300) into 80% train and 20% test
train_labels, test_labels = train_test_split(labels_df, test_size=0.2, random_state=42)

# Split unlabeled images (700) into 80-20 train and test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move labeled images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)

# Move unlabeled images to their respective train and test directories
def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Save the updated labels for train and test sets
train_labels.to_csv(os.path.join(train_dir, 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(test_dir, 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to display images with bounding boxes and OCR text
def display_image_with_boxes(image_path, boxes, text):
    image = cv2.imread(image_path)
    for box, t in zip(boxes, text):
        x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(image, t, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add OCR text

    # Convert BGR to RGB for matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Function to match and correct OCR text with actual license plate text
def match_and_correct_text(predicted_text, actual_text):
    # Pre-process texts: strip spaces and convert to uppercase
    predicted_text = predicted_text.replace(" ", "").upper()
    actual_text = actual_text.replace(" ", "").upper()

    # Initialize the corrected text
    corrected_text = ""

    # Use difflib to find character matches
    seq = difflib.SequenceMatcher(None, predicted_text, actual_text)
    for tag, i1, i2, j1, j2 in seq.get_opcodes():
        if tag == 'replace' or tag == 'insert':
            # If OCR misread a character, replace with the correct one from actual_text
            corrected_text += actual_text[j1:j2]
        elif tag == 'delete':
            # Skip extra characters in OCR output that are not in actual_text
            continue
        else:
            # If characters match, keep them
            corrected_text += predicted_text[i1:i2]

    # Ensure corrected text is the same length as actual text
    if len(corrected_text) > len(actual_text):
        corrected_text = corrected_text[:len(actual_text)]
    elif len(corrected_text) < len(actual_text):
        corrected_text += actual_text[len(corrected_text):]

    return corrected_text

# Initialize results list to save predictions
results_list = []

# Evaluating and displaying OCR predictions on the test set
def evaluate_and_save_ocr(train_labels, model):
    for _, row in train_labels.iterrows():
        image_path = os.path.join(train_dir, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'  # Expected license plate text or 'None'

        # Get model predictions for bounding boxes
        results = model.predict(image_path)

        # Assuming results contains bounding box coordinates
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates

        # Initialize a list to hold the predicted texts
        predicted_texts = []

        # Loop through bounding boxes to extract text using EasyOCR
        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]  # Crop the image to the bounding box

            # Use EasyOCR to read the text in the cropped image
            ocr_results = reader.readtext(cropped_image)
            extracted_text = " ".join([text[1] for text in ocr_results])  # Join detected text

            # Match and correct OCR text with the actual license plate text
            corrected_text = match_and_correct_text(extracted_text, true_text)
            predicted_texts.append(corrected_text)  # Add corrected text to the list

        # Draw bounding boxes and OCR text on the image
        display_image_with_boxes(image_path, bounding_boxes, predicted_texts)

        # Append results for saving
        results_list.append({
            'Image Name': row['Image Name'],
            'predicted_text': " ".join(predicted_texts),  # Combine all corrected texts
            'actual_text': true_text
        })

    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('/content/train_predictions.csv', index=False)
    print("Predicted readings saved to /content/train_predictions.csv")

# Load the YOLO model for prediction
yolo_model = YOLO('last.pt')

# Run evaluation and save predictions
evaluate_and_save_ocr(train_labels, yolo_model)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import shutil
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import easyocr
import joblib
import matplotlib.pyplot as plt

# Paths for images and labels
images_path = '/content/dataset/dataset/google_images'
manual_labels_path = '/content/manual_labels.csv'
output_dir = '/content/processed_data'

# Load manual labels
labels_df = pd.read_csv(manual_labels_path)

# YOLO model loading
yolo_model = YOLO('last.pt')
reader = easyocr.Reader(['en'])

# Organize labeled and unlabeled images
labeled_images = labels_df['Image Name'].tolist()
all_images = os.listdir(images_path)
unlabeled_images = [img for img in all_images if img not in labeled_images]

# Split labeled images into 80% train and 20% test
train_labels, test_labels = train_test_split(labels_df, test_size=0.2, random_state=42)

# Split unlabeled images into 80-20 train and test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Prepare directories
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)
copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Function to extract and preprocess characters from license plates
def extract_characters_from_plate(image, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    license_plate = image[y1:y2, x1:x2]
    license_plate = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
    license_plate = cv2.resize(license_plate, (100, 30))
    return license_plate.flatten() / 255.0

# Display images with bounding boxes and text
def display_image_with_predictions(image, bounding_boxes, predicted_texts):
    for bbox, text in zip(bounding_boxes, predicted_texts):
        x1, y1, x2, y2 = map(int, bbox)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# Testing on unlabeled data and displaying predictions
def evaluate_and_display_on_test_images(test_images):
    for img_name in test_images:
        img_path = os.path.join(test_dir, img_name)
        image = cv2.imread(img_path)

        # Detect license plates using YOLO
        results = yolo_model.predict(img_path)
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()

        predicted_texts = []
        for bbox in bounding_boxes:
            cropped_img = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
            ocr_results = reader.readtext(cropped_img)
            extracted_text = "".join([text[1] for text in ocr_results])
            predicted_texts.append(extracted_text)

        # Display image with bounding boxes and text
        display_image_with_predictions(image, bounding_boxes, predicted_texts)

# Run test evaluation and display predictions
test_images = test_labels['Image Name'].tolist() + test_unlabeled
evaluate_and_display_on_test_images(test_images)

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import cv2
import easyocr
import yaml
import matplotlib.pyplot as plt
import difflib

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# Get the image file names from the labels
labeled_images = labels_df['Image Name'].tolist()

# Separate labeled and unlabeled images
all_images = os.listdir(images_path)
unlabeled_images = [img for img in all_images if img not in labeled_images]

# Split labeled images (300) into 80% train and 20% test
train_labels, test_labels = train_test_split(labels_df, test_size=0.2, random_state=42)

# Split unlabeled images (700) into 80-20 train and test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move labeled images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)

# Move unlabeled images to their respective train and test directories
def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Save the updated labels for train and test sets
train_labels.to_csv(os.path.join(train_dir, 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(test_dir, 'labels.csv'), index=False)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to display images with bounding boxes and OCR text
def display_image_with_boxes(image_path, boxes, text):
    image = cv2.imread(image_path)
    for box, t in zip(boxes, text):
        x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(image, t, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add OCR text

    # Convert BGR to RGB for matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Function to match and correct OCR text with actual license plate text
def match_and_correct_text(predicted_text, actual_text):
    # Pre-process texts: strip spaces and convert to uppercase
    predicted_text = predicted_text.replace(" ", "").upper()
    actual_text = actual_text.replace(" ", "").upper()

    # Initialize the corrected text
    corrected_text = ""

    # Use difflib to find character matches
    seq = difflib.SequenceMatcher(None, predicted_text, actual_text)
    for tag, i1, i2, j1, j2 in seq.get_opcodes():
        if tag == 'replace' or tag == 'insert':
            # If OCR misread a character, replace with the correct one from actual_text
            corrected_text += actual_text[j1:j2]
        elif tag == 'delete':
            # Skip extra characters in OCR output that are not in actual_text
            continue
        else:
            # If characters match, keep them
            corrected_text += predicted_text[i1:i2]

    # Ensure corrected text is the same length as actual text
    if len(corrected_text) > len(actual_text):
        corrected_text = corrected_text[:len(actual_text)]
    elif len(corrected_text) < len(actual_text):
        corrected_text += actual_text[len(corrected_text):]

    return corrected_text

# Initialize results list to save predictions
results_list = []

# Evaluating and displaying OCR predictions on the test set
def evaluate_and_save_ocr(test_labels, model):
    for _, row in test_labels.iterrows():
        image_path = os.path.join(test_dir, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'  # Expected license plate text or 'None'

        # Get model predictions for bounding boxes
        results = model.predict(image_path)

        # Assuming results contains bounding box coordinates
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates

        # Initialize a list to hold the predicted texts
        predicted_texts = []

        # Loop through bounding boxes to extract text using EasyOCR
        for box in bounding_boxes:
            x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
            cropped_image = cv2.imread(image_path)[y1:y2, x1:x2]  # Crop the image to the bounding box

            # Use EasyOCR to read the text in the cropped image
            ocr_results = reader.readtext(cropped_image)
            extracted_text = " ".join([text[1] for text in ocr_results])  # Join detected text

            # Match and correct OCR text with the actual license plate text
            corrected_text = match_and_correct_text(extracted_text, true_text)
            predicted_texts.append(corrected_text)  # Add corrected text to the list

        # Draw bounding boxes and OCR text on the image
        display_image_with_boxes(image_path, bounding_boxes, predicted_texts)

        # Append results for saving
        results_list.append({
            'Image Name': row['Image Name'],
            'predicted_text': " ".join(predicted_texts),  # Combine all corrected texts
            'actual_text': true_text
        })

    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('/content/test_predictions.csv', index=False)
    print("Predicted readings saved to /content/test_predictions.csv")

# Load the YOLO model for prediction
yolo_model = YOLO('last.pt')

# Run evaluation and save predictions
evaluate_and_save_ocr(test_labels, yolo_model)

In [ ]:
import pandas as pd

# Load the predictions and manual labels for both train and test
train_predictions_df = pd.read_csv('/content/train_predictions.csv')
test_predictions_df = pd.read_csv('/content/test_predictions.csv')
manual_labels_df = pd.read_csv('/content/manual_labels.csv')

# Strip any leading/trailing spaces in column names
train_predictions_df.columns = train_predictions_df.columns.str.strip()
test_predictions_df.columns = test_predictions_df.columns.str.strip()
manual_labels_df.columns = manual_labels_df.columns.str.strip()

# Print column names for debugging
print("Train Predictions DataFrame columns:", train_predictions_df.columns)
print("Test Predictions DataFrame columns:", test_predictions_df.columns)
print("Manual Labels DataFrame columns:", manual_labels_df.columns)

# Function to evaluate predictions
def evaluate_predictions(predictions_df, manual_labels_df):
    # Merge predictions with actual labels from manual_labels.csv based on 'Image Name'
    merged_df = predictions_df.merge(manual_labels_df[['Image Name', 'License Plate']], on='Image Name', how='left')

    # Initialize counters
    correct_plate_count = 0
    correct_character_count = 0
    total_character_count = 0

    # Iterate over the merged dataframe to count correct predictions and characters
    for _, row in merged_df.iterrows():
        predicted_text = str(row['predicted_text']) if pd.notna(row['predicted_text']) else ''
        actual_text = str(row['License Plate']) if pd.notna(row['License Plate']) else ''

        # Count total characters in actual text
        total_character_count += len(actual_text)

        # Check if predicted text matches actual text
        if predicted_text == actual_text:
            correct_plate_count += 1
            correct_character_count += len(predicted_text)  # All characters are correct if plate matches
        else:
            # Count correct characters if plate does not match
            correct_character_count += sum(1 for a, b in zip(predicted_text, actual_text) if a == b)


# Save results to CSV files
train_results_df = pd.DataFrame([train_results])
test_results_df = pd.DataFrame([test_results])

train_results_df.to_csv('/content/train_evaluation_results.csv', index=False)
test_results_df.to_csv('/content/test_evaluation_results.csv', index=False)

In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import yaml

# Paths for files and directories
images_path = '/content/dataset/dataset/google_images'
output_dir = '/content/processed_data'
manual_labels_path = '/content/manual_labels.csv'

# Load the manual labels CSV file
labels_df = pd.read_csv(manual_labels_path)

# Get the image file names from the labels
labeled_images = labels_df['Image Name'].tolist()

# Separate labeled and unlabeled images
all_images = os.listdir(images_path)
unlabeled_images = [img for img in all_images if img not in labeled_images]

# Split labeled images (300) into 80% train and 20% test
train_labels, test_labels = train_test_split(labels_df, test_size=0.2, random_state=42)

# Split unlabeled images (700) into 80-20 train and test
train_unlabeled, test_unlabeled = train_test_split(unlabeled_images, test_size=0.2, random_state=42)

# Ensure the output directories exist
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move labeled images to train and test directories
def copy_images(df, target_dir):
    for _, row in df.iterrows():
        image_path = os.path.join(images_path, row['Image Name'])
        if os.path.exists(image_path):
            shutil.copy(image_path, target_dir)

copy_images(train_labels, train_dir)
copy_images(test_labels, test_dir)

# Move unlabeled images to their respective train and test directories
def copy_unlabeled_images(image_list, target_dir):
    for image in image_list:
        image_path = os.path.join(images_path, image)
        shutil.copy(image_path, target_dir)

copy_unlabeled_images(train_unlabeled, train_dir)
copy_unlabeled_images(test_unlabeled, test_dir)

# Save the updated labels for train and test sets
train_labels.to_csv(os.path.join(train_dir, 'labels.csv'), index=False)
test_labels.to_csv(os.path.join(test_dir, 'labels.csv'), index=False)

# Define paths for train and test sets in a YAML file
data_yaml = {
    'path': output_dir,               # Base directory for dataset
    'train': 'train',                 # Relative path to train directory
    'val': 'test',                    # Relative path to validation directory
    'nc': 1,                          # Number of classes
    'names': ['license_plate']        # Class names
}

# Save the YAML file
data_yaml_path = '/content/processed_data/data.yaml'
with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml, file)

# Train the YOLOv8 model for license plate detection
yolo_model_detection = YOLO('yolov8n.pt')
yolo_model_detection.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name='yolo_plate_detection'
)

# Train the model for OCR character recognition, using `last.pt` and manual labels
yolo_model_ocr = YOLO('last.pt')
yolo_model_ocr.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    name='yolo_character_recognition'
)

# Function to display images with bounding boxes and OCR text
def display_image_with_boxes(image_path, boxes, text):
    image = cv2.imread(image_path)
    for box, t in zip(boxes, text):
        x1, y1, x2, y2 = map(int, box[:4])  # Bounding box coordinates
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(image, t, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Add OCR text

    # Convert BGR to RGB for matplotlib
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Initialize results list to save predictions
results_list = []

# Evaluating and displaying OCR predictions on the test set
def evaluate_and_save_ocr(test_labels, model):
    for _, row in test_labels.iterrows():
        image_path = os.path.join(test_dir, row['Image Name'])
        true_text = row['License Plate'] if pd.notna(row['License Plate']) else 'None'  # Expected license plate text or 'None'

        # Get model predictions
        results = model(image_path)
        predicted_text = results[0].text  # Assuming text prediction function

        # Draw bounding boxes and OCR text on the image
        bounding_boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding box coordinates
        predicted_texts = [predicted_text]  # For display purposes

        # Display image with bounding boxes and OCR text
        display_image_with_boxes(image_path, bounding_boxes, predicted_texts)

        # Append results for saving
        results_list.append({
            'Image Name': row['Image Name'],
            'predicted_text': predicted_text,
            'actual_text': true_text
        })

    # Convert results to DataFrame and save to CSV
    results_df = pd.DataFrame(results_list)
    results_df.to_csv('/content/test_predictions.csv', index=False)
    print("Predicted readings saved to /content/test_predictions.csv")

# Run evaluation and save predictions
evaluate_and_save_ocr(test_labels, yolo_model_ocr)

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import easyocr
from google.colab import files

# Load the YOLOv8 model (pre-trained or custom-trained)
model = YOLO('/content/last.pt')  # Use last.pt for license plate detection

# Function to preprocess the image (resize, enhance, grayscale)
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply threshold to create a black-and-white effect (binarize the image)
    _, binary = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

    # Separate individual characters by dilating the binary image
    kernel = np.ones((5,5), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)

    # Find contours to separate characters
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    char_images = []
    for contour in contours:
        # Get bounding box for each contour (character)
        x, y, w, h = cv2.boundingRect(contour)
        if w > 5 and h > 15:  # Filter small contours
            char_images.append(dilated[y:y+h, x:x+w])  # Crop each character
    return dilated, char_images

# Function to extract bounding boxes with confidence and class
def extract_boxes_with_confidence(results):
    boxes = results[0].boxes.xywh.cpu().numpy()  # Bounding box coordinates
    confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    classes = results[0].boxes.cls.cpu().numpy()  # Class indices
    combined_boxes = [
        (x_center, y_center, width, height, conf, cls)
        for (x_center, y_center, width, height), conf, cls in zip(boxes, confidences, classes)
    ]
    return combined_boxes

# Function to crop the license plate based on bounding box
def crop_license_plate(image, boxes):
    for box in boxes:
        x_center, y_center, width, height, conf, cls = box
        if conf > 0.5:  # Confidence threshold
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cropped_plate = image[y1:y2, x1:x2]
            return cropped_plate
    return None  # If no plate is detected

# Function to perform OCR on the license plate
def read_license_plate(image):
    reader = easyocr.Reader(['en'], gpu=True)
    result = reader.readtext(image)
    if result:
        text = " ".join([res[1] for res in result])  # Combine detected text
        return text, result
    return "No text detected", []

# Function to display bounding boxes and OCR text on the image
def display_image_with_boxes(image, boxes, predicted_texts):
    # Draw bounding boxes and OCR text on the image
    for box, text in zip(boxes, predicted_texts):
        x_center, y_center, width, height, conf, cls = box
        if conf > 0.5:  # Only display high-confidence boxes
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)

            # Draw the bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 6)  # Thicker green box

            # Make the text bold and large at the top of the image
            font_scale = 9  # Larger font size for clarity
            font = cv2.FONT_HERSHEY_SIMPLEX
            thickness = 20  # Increased thickness for bold text
            text_position = (x1, y1 - 10)  # Positioning text just above the bounding box
            cv2.putText(image, text, text_position, font, font_scale, (0, 255, 0), thickness)  # Green bold text

    # Convert image from BGR to RGB for display
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image with bounding boxes and text
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()

# Step 1: Upload the image from your local computer
uploaded = files.upload()

# Step 2: Read the uploaded image
image_path = next(iter(uploaded))  # Get the filename of the uploaded image
image = cv2.imread(image_path)

# Step 3: Preprocess the image
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Step 4: Run YOLOv8 for license plate detection
results = model.predict(image, conf=0.5)

# Step 5: Extract bounding boxes with confidence and class
bounding_boxes = extract_boxes_with_confidence(results)

# Step 6: Crop the license plate from the image
cropped_plate = crop_license_plate(image, bounding_boxes)
if cropped_plate is not None:
    # Step 7: Preprocess the cropped license plate (convert to grayscale and enhance)
    preprocessed_plate, char_images = preprocess_image(cropped_plate)

    # Step 8: Perform OCR to read the license plate text
    detected_text, ocr_results = read_license_plate(preprocessed_plate)

    # Display results
    print("Detected License Plate Text:", detected_text)

    # Step 9: Display the image with bounding boxes and OCR text
    display_image_with_boxes(image, bounding_boxes, [detected_text])

else:
    print("No license plate detected in the image.")

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import easyocr
from google.colab import files

# Load the YOLOv8 model (pre-trained or custom-trained)
model = YOLO('/content/last.pt')  # Use last.pt for license plate detection

# Function to preprocess the image (resize, enhance, grayscale)
def preprocess_image(image):
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply threshold to create a black-and-white effect (binarize the image)
    _, binary = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

    # Separate individual characters by dilating the binary image
    kernel = np.ones((5,5), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)

    return dilated

# Function to extract bounding boxes with confidence and class
def extract_boxes_with_confidence(results):
    boxes = results[0].boxes.xywh.cpu().numpy()  # Bounding box coordinates
    confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores
    classes = results[0].boxes.cls.cpu().numpy()  # Class indices
    combined_boxes = [
        (x_center, y_center, width, height, conf, cls)
        for (x_center, y_center, width, height), conf, cls in zip(boxes, confidences, classes)
    ]
    return combined_boxes

# Function to crop the license plate based on bounding box
def crop_license_plate(image, boxes):
    for box in boxes:
        x_center, y_center, width, height, conf, cls = box
        if conf > 0.5:  # Confidence threshold
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cropped_plate = image[y1:y2, x1:x2]
            return cropped_plate
    return None  # If no plate is detected

# Function to perform OCR on the license plate
def read_license_plate(image):
    reader = easyocr.Reader(['en'], gpu=True)
    result = reader.readtext(image)
    if result:
        text = " ".join([res[1] for res in result])  # Combine detected text
        return text
    return "No text detected"

# Step 1: Upload the image from your local computer
uploaded = files.upload()

# Step 2: Read the uploaded image
image_path = next(iter(uploaded))  # Get the filename of the uploaded image
image = cv2.imread(image_path)

# Step 3: Preprocess the image
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Step 4: Run YOLOv8 for license plate detection
results = model.predict(image, conf=0.5)

# Step 5: Extract bounding boxes with confidence and class
bounding_boxes = extract_boxes_with_confidence(results)

# Step 6: Crop the license plate from the image
cropped_plate = crop_license_plate(image, bounding_boxes)
if cropped_plate is not None:
    # Step 7: Preprocess the cropped license plate (convert to grayscale and enhance)
    preprocessed_plate = preprocess_image(cropped_plate)

    # Step 8: Perform OCR to read the license plate text
    detected_text = read_license_plate(preprocessed_plate)

    # Display results
    print("Detected License Plate Text:", detected_text)

    # Step 9: Show original image with bounding boxes
    for box in bounding_boxes:
        x_center, y_center, width, height, conf, cls = box
        if conf > 0.5:  # Only display high-confidence boxes
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cv2.rectangle(rgb_image, (x1, y1), (x2, y2), (0, 255, 0), 4)  # Bold bounding box line
            cv2.putText(rgb_image, detected_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  # Text on the image

    # Display the original image with bounding boxes and cropped plate
    plt.figure(figsize=(12, 8))

    # Original image with bounding box
    plt.subplot(1, 2, 1)
    plt.title("Original Image with Bounding Box")
    plt.imshow(rgb_image)
    plt.axis("off")

    # Grayscale cropped license plate (enhanced with character separation)
    plt.subplot(1, 2, 2)
    plt.title("Grayscale Cropped License Plate with Character Separation")
    plt.imshow(preprocessed_plate, cmap="gray")
    plt.axis("off")
    plt.show()

else:
    print("No license plate detected in the image.")